**각 Top Features만을 사용한 RandomForestClassifier와 LSTM의 성능 비교**

In [1]:
import numpy as np, os
import pandas as pd

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns

import tensorflow as tf
from tensorflow import keras

import warnings
warnings.filterwarnings(action='ignore')

#한글설정
import matplotlib.font_manager as fm

font_dirs = ['/usr/share/fonts/truetype/nanum', ]
font_files = fm.findSystemFonts(fontpaths=font_dirs)

for font_file in font_files:
    fm.fontManager.addfont(font_file)
    
# 한글 출력을 위해서 폰트 옵션을 설정합니다.
# "axes.unicode_minus" : 마이너스가 깨질 것을 방지

sns.set(font="NanumBarunGothic", 
        rc={"axes.unicode_minus":False},
        style='darkgrid')

# RandomForest
1. Feature Importance를 구하여 Top Features 추출
2. Top Features만을 사용한 데이터로 RandomForestClassifier 학습 후 정확도 측정

In [22]:
COLS = list(pd.read_csv('/project/LSH/total_data_7727.csv')['ITEMID'].sort_values().unique())

In [9]:
import random
random.seed(42)
path = '/project/LSH/'
x =  np.load(path + 'x_(7727,4069).npy')
y = np.load(path + 'y_(7727,1).npy')

idx = list(range(len(x)))
random.shuffle(idx)

i = round(x.shape[0]*0.8)
X_train, y_train = x[idx[:i],:], y[idx[:i]]
X_test, y_test = x[idx[i:],:], y[idx[i:]]

In [10]:
X.shape

(7727, 4069)

In [14]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics 

model = RandomForestClassifier()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print(f'정확도 : {metrics.accuracy_score(y_test, pred)}')

정확도 : 0.7320388349514563


In [16]:
importances = model.feature_importances_

In [21]:
sort = np.argsort(importances)[::-1]

In [26]:
result = []
for f in range(X_test.shape[1]-1):
    result.append({'rank' : f+1, 'feature' : COLS[f], 'importances' : importances[sort[f]]})

In [35]:
rf_fi = pd.DataFrame(result).sort_values('importances', ascending=False)
rf_fi

,rank,feature,importances
0,1,0,0.016167
1,2,50803,0.010767
2,3,50804,0.010180
3,4,50805,0.006368
4,5,50806,0.006270
...,...,...,...
3386,3387,54569170102,0.000000
3387,3388,54569189502,0.000000
3388,3389,54569204000,0.000000
3389,3390,54569241100,0.000000
